<p style="font-weight:bold;"> <span style="font-size: 36px"> Initialize Data </span> </p>

In [ ]:
#!import "../ifrs17/Import/Importers"
//#!import "//ifrs17/dev/Import/Importers" 

In [ ]:
#!import "../ifrs17/CalculationEngine"

---

In [ ]:
// not working because enums are not supported (also this requires the excel cells format to be tuned adequately)
//await Domain.GenerateFromFile("Dimensions.xlsx").ExecuteAsync()

In [ ]:
/*
var domain = Domain.CreateDomain("Dimension")
                .WithType<Novelty>()            .WithType<AocType>()                    .WithType<PnlVariableType>()            .WithType<AocConfiguration>()
                .WithType<PvAmountType>()       .WithType<DeferrableAmountType>()       .WithType<CalculationType>()            .WithType<RiskDriver>()
                .WithType<EconomicBasis>()      .WithType<EstimateType>()               .WithType<ValuationApproach>()          .WithType<LineOfBusiness>()
                .WithType<OciType>()            .WithType<LiabilityType>()              .WithType<Profitability>()              .WithType<Currency>()
                .WithType<ReportingNode>()      .WithType<Partner>()                    .WithType<CreditRiskRating>()           .WithType<Scenario>()
                .WithType<ProjectionConfiguration>()    
                .WithType<PartitionByReportingNode>()            // TODO: automatize partitions
                .WithType<PartitionByReportingNodeAndPeriod>()   // TODO: not working cause the name is too long for excel
                .ToDomain();
*/

In [ ]:
// not usable at the moment for the reason outlined above
//await Import.FromFile("Dimensions.xlsx").WithDomain(domain).WithTarget(DataSource).ExecuteAsync()

In [ ]:
// FEEDBACK: use domain (e.g. IFRS17.Import), see above
await Import.FromFile("Dimensions.xlsx")
            .WithType<Novelty>()                  .WithType<AocType>()                   .WithType<PnlVariableType>()            .WithType<AocConfiguration>()
            .WithType<PvAmountType>()             .WithType<DeferrableAmountType>()      //.WithType<CalculationType>()            
            .WithType<RiskDriver>()
            .WithType<EconomicBasis>()            .WithType<EstimateType>()              .WithType<ValuationApproach>()          .WithType<LineOfBusiness>()
            .WithType<OciType>()                  .WithType<LiabilityType>()             .WithType<Profitability>()              .WithType<Currency>()
            .WithType<ReportingNode>()            .WithType<Partner>()                   .WithType<CreditRiskRating>()           .WithType<Scenario>()
            .WithType<ProjectionConfiguration>()
            .WithType<PartitionByReportingNode>()                                                            // TODO: automatize partitions
            .WithType<PartitionByReportingNodeAndPeriod>("PartitionByReportingNodeAndPeri")                  // TODO: ugly
            .WithTarget(DataSource)
            .ExecuteAsync()

In [ ]:
await Import.FromFile("DataNodes.xlsx").WithFormat("DataNode").WithTarget(DataSource).ExecuteAsync()

In [ ]:
// TODO: set the imported DataNodes to Active by default
//await UploadDataNodeStateAsync("DataNodeStates.csv")

In [ ]:
// TODO: 1. map template not working, 2. set default values
//await UploadDataNodeParameterAsync("DataNodeParameters.csv")